<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_04_YogFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ॐ श्री गणेशाय नमः

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#Retrieving Charts that contain certain Yogs ( or patterns)



For those who may not have access to a live MongoDB instance, we have set up a mechanism where the JSON file that would have been loaded into a hosted MongoDB instance, will be loaded into a locally (that is in the VM) installed MongoDB instance and the queries can be run in a manner that is identical to a live, hosted MongoDB instance. <br>
However the option to access a remote, persistent database is also available.<br><br>
For full information about this project, please see the [Parashar21](https://prithwis.github.io/parashar21/) github page.

#SetUp Environment

##Install Software

In [1]:
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!/usr/bin/python3 -m pip -qq install "pymongo[srv]" # required to access MongoDB Atlas Database servers
#!apt install mongodb > /dev/null            # install local MongoDB server
#!service mongodb start                     # start Local MongoDB server instance only if using Local Mongo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [2]:
# Patch for MongoDB error issues
# --------------------
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg \
   --dearmor
# --------------------
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
# --------------------
!sudo apt-get update > /dev/null
# --------------------
!sudo apt-get install -y mongodb-org > /dev/null          # error, warning messages may be ignored
# --------------------
!sudo mkdir -p /var/log/mongodb /var/lib/mongodb
# --------------------
!sudo chown `whoami` /var/lib/mongodb /var/log/mongodb/

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [3]:
#!service mongodb start                                      # start Local MongoDB server instance --- does not work anymore
!nohup mongod --config /etc/mongod.conf &                    # patch for manual start

nohup: appending output to 'nohup.out'


##Install, import prerequisites

In [4]:
#Utility functions
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

from google.colab import files

#import matplotlib.pyplot as plt
#import math


In [5]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py
# ---------------------------

!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png
!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png

In [6]:
import p21
import p21utils
import p21utilsR
import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
importlib.reload(p21YogInfo)

10 Yogs on record
0 : Amala Yog | 10th from Moon OR Lagna should have a benefic
1 : RajLakshana Yog | Jupiter, Venus, Mercury, Moon should be in Lagna or they should be in a Kendra
2 : Chatussagara Yog | All the Kendras 1/4/7/10 are occupied by planets, Ra, Ke not counted here
3 : Adhi678 Yog | Benefics located in 6/7/8th  from Moon
4 : Chandra Mangal Yog | Mars Conjoins Moon or  Mars and Moon in Mutual Aspect
5 : Kemadruma Yog : No planets on either sides of the Moon
6 : Anapha Yog |Planets other than Sun in 12th Place from Moon
7 : Sunapha Yog |Planets other than Sun in 2nd Place from Moon
8 : Dhurdhura Yog | Planets on both sides of the Moon
9 : Gajakesari Yog | Jupiter is in Kendra or 1,4,7,10th from Moon
10 Yogs on record
0 : Amala Yog | 10th from Moon OR Lagna should have a benefic
1 : RajLakshana Yog | Jupiter, Venus, Mercury, Moon should be in Lagna or they should be in a Kendra
2 : Chatussagara Yog | All the Kendras 1/4/7/10 are occupied by planets, Ra, Ke not counted here
3 :

<module 'p21YogInfo' from '/content/p21YogInfo.py'>

#Two Options : Local MongoDB or Remote Atlas MongoDB

##Option 1 : Local MongoDB
This is the preferred option for anyone using application for the first time

In [7]:
#!service mongodb start                      # start Local MongoDB server instance

In [8]:
mongoURI = "mongodb://localhost:27017/"                               # for remote server
client = MongoClient(mongoURI)

db = client.parashar21
#pd.DataFrame(list(db.list_collections()))

###Download Horoscope data in JSON and load into MongoDB
Download Pre-computed JSON file containing 39662 charts from Rodden Database and import the same into the local MongoDB datbase

In [9]:
# Retrieve JSON file from Github and load in local Mongo
#!wget -O peopleData.json -q https://github.com/prithwis/parashar21/raw/main/data/peopleDataTest5.json

!gdown  -O peopleData.json 1GIPjxHenAm2EKdtx8T0_RwkzkmCRGEH7  # 39662 documents
!mongoimport --db parashar21 --collection khona21 --drop --file peopleData.json

Downloading...
From: https://drive.google.com/uc?id=1GIPjxHenAm2EKdtx8T0_RwkzkmCRGEH7
To: /content/peopleData.json
100% 143M/143M [00:02<00:00, 62.0MB/s]
2023-07-25T14:47:27.356+0000	connected to: mongodb://localhost/
2023-07-25T14:47:27.357+0000	dropping: parashar21.khona21
2023-07-25T14:47:30.362+0000	[##......................] parashar21.khona21	11.7MB/137MB (8.6%)
2023-07-25T14:47:33.358+0000	[###.....................] parashar21.khona21	18.1MB/137MB (13.3%)
2023-07-25T14:47:36.357+0000	[####....................] parashar21.khona21	27.2MB/137MB (19.9%)
2023-07-25T14:47:39.357+0000	[#######.................] parashar21.khona21	41.1MB/137MB (30.1%)
2023-07-25T14:47:42.357+0000	[#########...............] parashar21.khona21	54.9MB/137MB (40.2%)
2023-07-25T14:47:45.357+0000	[############............] parashar21.khona21	68.8MB/137MB (50.4%)
2023-07-25T14:47:48.357+0000	[#############...........] parashar21.khona21	75.6MB/137MB (55.3%)
2023-07-25T14:47:51.357+0000	[##############.........

In [10]:
p21.kollection = db.khona21                   # Set the collection to work with

totalCharts = p21.kollection.count_documents({})
dbSize = db.command( {"dbStats": 1, "scale": 1024 } )['dataSize']
print(totalCharts, ' documents,',round(dbSize,2), " KB data")

39662  documents, 146582.36  KB data


##Option 2 : Remote Atlast Database
https://account.mongodb.com/account/login <br>
Do not attempt to run these cells if you do not have (a) access to a remote hosted MongoDB database and (b) credential files stored in your G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# -------------------------------------------
# copy credentials file
!cp /content/drive/'My Drive'/Praxis/WebCredentials/Parashar21_Atlas_18Sep22.py credentials.py
# -------------------------------------------
# MongoDB credentials are stored here
#
from credentials import parashar21URI
# -------------------------------------------
mongoURI = parashar21URI                                   # for remote ATlas server

client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']
# ----------------------------------------------

Mounted at /content/drive


['Parashar21', 'admin', 'local']

In [ ]:
# Show existing Collections
db = client.Parashar21
#pd.DataFrame(list(db.list_collections()))
#kollection = db.khona21Test                   # Set the collection to work with
p21.kollection = db.khona21                   # Set the collection to work with
totalCharts = p21.kollection.count_documents({})
print(totalCharts)

39662


In [ ]:
#print(p21.kollection)

Collection(Database(MongoClient(host=['cluster0-shard-00-01.yarln.mongodb.net:27017', 'cluster0-shard-00-00.yarln.mongodb.net:27017', 'cluster0-shard-00-02.yarln.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-djz97v-shard-0', tls=True), 'Parashar21'), 'khona21')


#Report Generation Function
Please note that the generated report MS Office doc format **will be stored in the Colab VM drive** and needs to be downloaded to local machine for viewing

In [11]:
def genReport(yText, ChartStyle,Sample = True):

    if Sample:
        df4Report = df.sample(5)
    else:
        df4Report = df

    selectedCharts = len(df)
    printCharts = len(df4Report)
    #printStatus = str(printCharts)+' charts of total '+str(totalCharts)+' printed.'
    printStatus = str(len(df))+' charts of total '+str(totalCharts)+' selected and '+str(printCharts)+' printed'



    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21Yog_'+yText[0:4]+'_'+ChartStyle[0:1]+'_'+now.strftime("%H%M%S")+'.doc'
    curQueryStr = json.dumps(selCond)[0:100]
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(yText,printStatus)                      # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    for chart in range(len(df4Report)):
        chartData =df4Report.iloc[chart]
        #R51_processChart(chartData)
        p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                                              # Creates two primary dicts, GLon, GRet
        p21utilsR.R30_LocateYogs()

        for ctype in ['Rashi','Navamsa']:
        #for ctype in ['Rashi']:
            p21.ChartType = ctype
            p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                                # Rashi Num as well as Rashi name
            if ChartStyle == 'Bengal':
                p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
            if ChartStyle == 'South':
                p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
            if ChartStyle == 'North':
                p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
            #


        #print(p21.GrahaBhava)
        #print(p21.LordBhav)


        p21utilsR.R512_FormatPage()
        #print('----')
    # ------------------------------------
    p21.document.save(fileName)
    return fileName

print('Tested on ',datetime.now(pytz.timezone('Asia/Kolkata')))

Tested on  2023-07-25 20:18:43.280603+05:30


#Retrieve Charts



##Yogs idenfied by BV Raman
300 Important Combinations

In [12]:
%%capture
yogName = 'Amala'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
if (len(df) > 0):
    ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')
else:
    print('no charts found')

In [13]:
%%capture
yogName = 'RajLakshana'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
if (len(df) > 0):
    ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')
else:
    print('no charts found')

In [ ]:
%%capture
yogName = 'Chatussagara'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
if (len(df) > 0):
    ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')
else:
    print('no charts found')

In [ ]:
%%capture
yogName = 'Adhi678'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
if (len(df) > 0):
    ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')
else:
    print('no charts found')

In [ ]:
%%capture
yogName = 'ChandraMangal'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'ChandraMangal2'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'ChandraMangal1'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Kemadruma'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Dhurdhura'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Anapha'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
yogName = 'Sunapha'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
%%capture
#from p21YogInfo import  yogCond, yogText
yogName = 'Gajakesari'
selCond = p21YogInfo.yogCond[yogName]
df = pd.DataFrame(list(p21.kollection.find(p21YogInfo.yogCond[yogName],p21.selCols)))
ReportFile = genReport(p21YogInfo.yogText[yogName],'Bengal')

In [ ]:
#
# Download last Report File
#
files.download(ReportFile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Random Test Patterns
These were initially used in the **P21_45_02_Pull_Print** program to test the query features of the MongoDB query language and have little astrological significance

In [ ]:
%%capture
selCond = {'GAspectedBy2.La': {'$in': ['Sa']}}                      # La aspected by Sa
df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
tag = 'Testing : '+json.dumps(selCond)
ReportFile = genReport(tag,'Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}}       # La aspected by Sa

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
tag = 'Testing : '+json.dumps(selCond)
ReportFile = genReport(tag,'Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa
                        {'GConjunctsG2.Su' : {'$in': ['Me']}}       # Su conjunct Me

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
tag = 'Testing : '+json.dumps(selCond)
ReportFile = genReport(tag,'North')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me
                        {'GrahaBhava.Mo' : {'$eq': 1}}              # Mo in First House
                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
tag = 'Testing : '+json.dumps(selCond)
ReportFile = genReport(tag,'North')

In [14]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me
                        {'GrahaBhava.Mo' : {'$eq': 1}} ,            # Mo in First House
                        {'LordBhav.4' : {'$eq': 5}}                 # 4th Lord in 5th House
                    ]
}
df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
tag = 'Testing : '+json.dumps(selCond)
ReportFile = genReport(tag,'Bengal',False)

In [ ]:
#
# Download last Report File
#
files.download(ReportFile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Clean up
!rm *.doc

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)

#Old, Left Over stuff

## zBudhaditya Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'BudhAditya'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genReport(yogText[yogName],'Bengal')

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'BudhAdityaNR'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genReport(yogText[yogName],'Bengal')

##zRaj Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'Rajyog'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genReport(yogText[yogName],'Bengal')

##Chandra Mangal (Mahalakshmi) Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'ChandraMangal2'
#yogName = 'ChandraMangal1' # this returns no records in our database
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genReport(yogText[yogName],'Bengal')

##Dhan Yog

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
yogName = 'DhanYog'
selCond = yogCond[yogName]
df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
genReport(yogText[yogName],'Bengal')

##All Yogs

In [ ]:
%%capture
from p21YogInfo import  yogCond, yogText
#yogList = ['BudhAditya','BudhAdityaNR', 'Gajakesari', 'Rajyog', 'ChandraMangal2','DhanYog']
for yogName in list(p21YogInfo.yogCond.keys()):
    #selCond = yogCond[yogName]
    df = pd.DataFrame(list(kollection.find(yogCond[yogName],p21.selCols)))
    #print(yogName, len(df))
    if len(df) > 0:
        genReport(yogText[yogName],'Bengal')


##Tests

In [ ]:
%%capture
yogText = 'Dhan^ yog : Lords 1 with 2 ^ ref : LHAE KN Rao, pg 77'
selCond = { 'BLConjunctsBL2.1': {'$in': [2,3]}        }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genReport(yogText,'Bengal')

In [ ]:
%%capture
yogText = 'Dummy^ yog : Ma not conjuct with Moon ^ ref : https://www.astroyogi.com/kundli/yog/mahalaxmi'
selCond = {"$and":
                    [
                        {'GConjunctsG2.Ma' : {'$in': ['Su']}} ,     # Ma not conjunct Mo
                        {'GConjunctsG2.Ma' : {'$nin': ['Mo']}} ,     # Ma not conjunct Mo
                        {'GrahaBhava.Mo' : {'$nin': [1,2,3,4,5,6,7,8,9,10]}}     # Mo not in 3,6,8,12

                    ]
         }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genReport(yogText,'Bengal')

In [ ]:
%%capture
yogText = 'Dummy^ yog : Lords 1 with 2,3 ^ ref : LHAE KN Rao, pg 77'
selCond = { 'BLConjunctsBL2.1': {'$in': ['2','3']} }

df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
genReport(yogText,'Bengal')

In [ ]:
!rm *.doc